In [ ]:
import pandas as pd

# Load transactions data
I logged in to Mint and downloaded the transactions as csv. It contains duplicates (almost half of them)

In [ ]:
df = pd.read_csv('data/transactions.csv')
df.head()

# Clean up

In [ ]:
def clean_transactions(df, d):
    df_all_trans = df[df['Date'] == d].reset_index(drop=True)
    df_all_trans['idx'] = df_all_trans.index.tolist()
    new_list = []
    ignore_idxs = []
    for idx, row in df_all_trans.iterrows():
        if idx in ignore_idxs:
            continue
        new_list.append(row.to_dict())
        amount = row['Amount']
        account_name = row['Account Name']
        trans_type = row['Transaction Type']
        category = row['Category']
        df_same_amounts = df_all_trans[
                    (df_all_trans['Amount'] == amount) & 
                    (df_all_trans['Account Name'] == account_name) & 
                    (df_all_trans['Transaction Type'] == trans_type) & 
                    # (df_all_trans['Category'] == category) & 
                    (df_all_trans['idx'] != idx)]
        indices = df_same_amounts.index.tolist()
        ignore_idxs.extend(indices)
    ignored_list = df_all_trans[df_all_trans['idx'].isin(ignore_idxs)].to_dict(orient='records')
    return new_list, ignored_list

In [ ]:
unique_dates = df['Date'].unique().tolist()
all_unique_trans = []
all_ignored_trans = []
for d in unique_dates:
    transacations, ignored_trans = clean_transactions(df, d)
    all_unique_trans.extend(transacations)
    all_ignored_trans.extend(ignored_trans)

# Unique transactions

In [ ]:
df_new = pd.DataFrame(all_unique_trans)
del df_new['idx']
df_new.head()

In [ ]:
print(df.size, df_new.size)

# Ignored transactions
To double check

In [ ]:
df_ignored = pd.DataFrame(all_ignored_trans)
df_ignored.head(100)

In [ ]:
df[df['Date'] == '10/11/2023']

# Save fixed transactions

Ready to be uploaded to Monarch

In [ ]:
df_new.to_csv('data/fixed_transactions.csv', index=False)